In [1]:
import pandas as pd
import numpy as np
import math

from sklearn import svm

In [2]:
training = pd.read_csv('../data/train/training.csv')
test = pd.read_csv('../data/test/sorted_test.csv')

training.head()
test.head()

# removing CO2 Bands
training = training.drop(training.ix[:,2655:2670].columns,axis=1)
test= test.drop(training.ix[:,2655:2670].columns,axis=1)

training[['SOC', 'pH', 'Ca', 'P', 'Sand']].describe()
test.describe()


,PIDN,m7497.96,m7496.04,m7494.11,m7492.18,m7490.25,m7488.32,m7486.39,m7484.46,m7482.54,...,REF7,RELI,TMAP,TMFI,Depth,Ca,P,pH,SOC,Sand
0,XNhoFZW5,0.302553,0.301137,0.299748,0.300354,0.302679,0.303799,0.301702,0.298936,0.298126,...,-0.646673,1.687734,0.190708,0.056843,Topsoil,-0.295749,-0.041336,-1.129366,0.353258,1.269748
1,9XNspFTd,0.270192,0.268555,0.266964,0.267938,0.271013,0.272346,0.269870,0.266976,0.266544,...,-0.646673,1.687734,0.190708,0.056843,Subsoil,-0.387442,-0.231552,-1.531538,-0.264023,1.692209
2,WDId41qG,0.317433,0.316265,0.314948,0.315224,0.316942,0.317764,0.316067,0.313874,0.313301,...,-0.814516,1.806660,0.190708,0.056843,Topsoil,-0.248601,-0.224635,-0.259551,0.064152,2.091835
3,JrrJf1mN,0.261116,0.259767,0.258384,0.259001,0.261310,0.262417,0.260534,0.258039,0.257246,...,-0.814516,1.806660,0.190708,0.056843,Subsoil,-0.332195,-0.318014,-0.577548,-0.318719,2.118477
4,ZoIitegA,0.260038,0.258425,0.256544,0.257030,0.259602,0.260786,0.258717,0.256352,0.255902,...,-0.780242,0.430513,0.190708,0.056843,Topsoil,-0.438350,-0.010210,-0.699135,-0.310905,2.164148


,PIDN,m7497.96,m7496.04,m7494.11,m7492.18,m7490.25,m7488.32,m7486.39,m7484.46,m7482.54,...,LSTD,LSTN,REF1,REF2,REF3,REF7,RELI,TMAP,TMFI,Depth
0,09gt9UK5,0.210899,0.209221,0.207645,0.208714,0.211972,0.213562,0.211267,0.208248,0.207341,...,-0.009394,0.197885,-0.622545,-0.548238,-0.637681,-0.413306,-0.130139,0.154013,-0.033905,Subsoil
1,0BVvxJ6a,0.191650,0.184120,0.177070,0.180624,0.192343,0.197502,0.187655,0.175447,0.172639,...,-0.277944,-0.851643,-0.792000,-1.058442,-0.792961,-0.785786,-0.588855,0.106200,0.559694,Subsoil
2,0KbdgApg,0.083966,0.074740,0.065596,0.069924,0.085145,0.092527,0.080625,0.064768,0.060597,...,-0.443809,-0.628839,-0.754909,-0.926716,-0.865424,-0.607359,-0.334013,-0.021042,0.182431,Topsoil
3,0MnuuduB,0.345962,0.338567,0.331786,0.336948,0.350820,0.356365,0.345285,0.332233,0.328589,...,-0.758853,-0.961080,-0.729455,-0.780148,-0.716356,-0.552923,-0.079171,0.394936,0.774477,Topsoil
4,0PcIsF9z,0.239019,0.232611,0.226266,0.229921,0.241821,0.247756,0.239242,0.227977,0.224762,...,-1.228289,1.156914,-0.696727,0.080705,0.118012,-0.920363,-0.266055,1.612625,1.199048,Subsoil


/home/sagarkar10/miniconda3/envs/mlpb/lib/python3.5/site-packages/ipykernel/__main__.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
/home/sagarkar10/miniconda3/envs/mlpb/lib/python3.5/site-packages/ipykernel/__main__.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


,SOC,pH,Ca,P,Sand
count,1157.000000,1157.000000,1157.000000,1157.000000,1157.000000
mean,0.080414,-0.028543,0.006442,-0.014524,-0.012646
std,1.141989,0.920224,1.070541,0.995469,0.988520
min,-0.857863,-1.886946,-0.535828,-0.418309,-1.493378
25%,-0.615639,-0.717841,-0.451077,-0.345681,-0.899649
50%,-0.349974,-0.175376,-0.348682,-0.269595,-0.134651
75%,0.275121,0.376442,-0.042654,-0.089755,0.786391
max,7.619989,3.416117,9.645815,13.266841,2.251685


,m7497.96,m7496.04,m7494.11,m7492.18,m7490.25,m7488.32,m7486.39,m7484.46,m7482.54,m7480.61,...,EVI,LSTD,LSTN,REF1,REF2,REF3,REF7,RELI,TMAP,TMFI
count,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,...,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000
mean,0.232165,0.226979,0.222080,0.225511,0.234976,0.238725,0.231077,0.222129,0.219685,0.219343,...,0.583861,-0.364974,0.269816,-0.668341,-0.572728,-0.594943,-0.640463,0.084040,0.545331,0.703120
std,0.098487,0.099179,0.099853,0.099608,0.098558,0.097966,0.098652,0.099714,0.099947,0.099900,...,0.727844,0.835115,0.883666,0.320982,0.350492,0.410649,0.364303,0.941628,0.682021,0.787634
min,-0.037277,-0.043984,-0.050737,-0.046731,-0.034096,-0.028093,-0.037383,-0.049268,-0.052309,-0.052265,...,-0.800292,-2.366282,-1.760197,-1.161455,-1.360853,-1.267081,-1.268649,-0.605844,-0.541529,-0.772670
25%,0.170926,0.165406,0.160249,0.163639,0.173564,0.177973,0.169524,0.160199,0.157818,0.157650,...,0.164723,-1.046907,-0.546596,-0.885818,-0.792208,-0.910973,-0.929435,-0.401971,-0.021042,0.182431
50%,0.246999,0.242504,0.239130,0.241215,0.249651,0.253969,0.245869,0.238573,0.236578,0.236108,...,0.507289,-0.390770,0.497065,-0.746909,-0.614100,-0.672878,-0.684476,-0.266055,0.501829,0.774477
75%,0.294928,0.290607,0.287255,0.290548,0.299234,0.301754,0.294658,0.286204,0.284108,0.283883,...,1.043003,0.261494,1.042551,-0.547273,-0.439703,-0.378882,-0.412550,0.124703,0.981483,1.053572
max,0.490711,0.486155,0.482087,0.485504,0.494250,0.497684,0.490965,0.483429,0.481667,0.481884,...,2.201895,1.666615,1.844185,0.572364,0.530612,0.681159,0.579637,5.731226,2.180246,2.622005


In [3]:
def numeric_depth(df):
    # Depth Converting to numeric.
    depth = []
    depth_dict = {}
    depth_dict['Topsoil']=1.0
    depth_dict['Subsoil']=0.0
    for i in df.index:
        depth_val = df.ix[i]['Depth']        
        depth.append(depth_dict[depth_val])
    df['Depth']=pd.Series(depth)
    return df

def change_index(df,colname):
    df = df.set_index(colname)
    df.index.names = [None]
    return df

def calculate_fit(model,valX,valY,pred,expec):
    mse = np.mean((pred-expec)**2)
    score = model.score(valX,valY)
    print (t)
    return mse,score    



In [4]:
training = numeric_depth(training)

In [5]:
test = numeric_depth(test)

/home/sagarkar10/miniconda3/envs/mlpb/lib/python3.5/site-packages/ipykernel/__main__.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


In [6]:
# train data for each target variable

feature_columns = [i for i in training.columns if i not in ['PIDN','SOC','pH','Ca','P','Sand']]

X = training[feature_columns]

Y={}
Y['SOC'] = training[['SOC']]
Y['pH'] =  training[['pH']]
Y['Ca'] = training[['Ca']]
Y['P'] = training[['P']]
Y['Sand'] = training[['Sand']]

# df to numpy matrices

X = X.as_matrix()

for k in Y:
    Y[k]=Y[k].as_matrix()


trainN = int(math.ceil(abs(0.8*X.shape[0])))
trainX = X[:trainN,:]
trainY = {}
for k in Y:
    trainY[k] = Y[k][:trainN]

validX = X[trainN+1:,:]

validY={}
for k in Y:
    validY[k] = Y[k][trainN+1:]

##Processing test data

In [ ]:
test = change_index(test,'PIDN')

### Making sure that there are no more string based fields

test[[i for i in test.columns if i.isalpha()]].describe()



,BSAN,BSAS,BSAV,CTI,ELEV,EVI,LSTD,LSTN,RELI,TMAP,TMFI,Depth
count,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000
mean,-0.617497,-0.639852,-0.671307,-0.108035,0.227325,0.583861,-0.364974,0.269816,0.084040,0.545331,0.703120,0.506190
std,0.256327,0.276273,0.307626,0.791396,1.152433,0.727844,0.835115,0.883666,0.941628,0.682021,0.787634,0.500306
min,-1.000000,-1.036364,-1.142217,-1.001026,-1.395768,-0.800292,-2.366282,-1.760197,-0.605844,-0.541529,-0.772670,0.000000
25%,-0.789855,-0.818182,-0.873460,-0.477960,-0.870156,0.164723,-1.046907,-0.546596,-0.401971,-0.021042,0.182431,0.000000
50%,-0.673913,-0.718182,-0.739082,-0.321260,0.336971,0.507289,-0.390770,0.497065,-0.266055,0.501829,0.774477,1.000000
75%,-0.528986,-0.536364,-0.537514,-0.016621,1.205568,1.043003,0.261494,1.042551,0.124703,0.981483,1.053572,1.000000
max,0.391304,0.436364,0.403136,3.596526,2.646548,2.201895,1.666615,1.844185,5.731226,2.180246,2.622005,1.000000


In [ ]:
# model - SVR
targets = ['SOC', 'pH', 'Ca', 'P', 'Sand']
mod={}
mse_list=[]
score_list=[]
for t in targets:
    mod[t] = svm.SVR(kernel='linear',C=1.0,epsilon=0.2)
    mod[t].fit(trainX,trainY[t].ravel())
    expec = validY[t]
    pred = mod[t].predict(validX)
    mse,score = calculate_fit(mod[t],validX,validY[t],pred,expec)
    mse_list.append(mse)
    score_list.append(score)
print(mse_list,score_list) 


output_matrix = test.copy()
preds={}
testX = output_matrix.as_matrix()
for t in targets: 
    t_pred = mod[t].predict(testX) 
    preds[t] = pd.DataFrame(list(t_pred.ravel()),columns=[t])

#Generating a single dataframe for submission
output = pd.concat([preds[k] for k in preds],axis=1)
output = output.set_index(test.index)

In [ ]:
output.index.name = 'PIDN'

In [ ]:
output.to_csv('svm_no_co2_final.csv',index_label='PIDN')